# LLM Mastery


## 1.Import all packages

In [36]:
from datetime import datetime
from dotenv import load_dotenv
import io
import ipdb
import os
import pdb
import platform
import requests
import sentencepiece as spm
import shutil
import sys
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm
import wandb
import zipfile

## 2.clear cache

In [37]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

torch.cuda.empty_cache()

import gc
# del tensor_variable
gc.collect()

23551

In [38]:
# downlaod necessary files
# wiki.txt
# wiki_tokenizer.model
# wiki_tokenizer.vocab
# encoded_date.pt

# files_url = 'https://ideami.com/llm_train'
# print ('Downloading file')
# response = requests.get(files_url)
# zipfile.ZipFile(io.BytesIO(response.content)).extractall('.')


In [39]:
# !pip install -q pdb

In [40]:
# def calculate (v1, v2):
#     ipdb.set_trace()
#     # pdb.set_trace()
#     calc1 = v1 *55
#     calc2 = calc1 * v2
#     return calc2

# print (calculate (2, 3))

## Initialize parameters

In [41]:
# architecture parameters
batch_size = 8
context = 512
embed_size = 384
n_layers = 7
n_heads = 7
BIAS = True

# Hyperparameters
lr = 3e-4
dropout = 0.05
weight_decay = 0.01
grad_clip = 1.0

# Training parmters
train_iters = 100000
eval_interval = 50
eval_iters = 10
compile = False
checkpoint_dir = 'models'
checkpoint_fn = 'latest.pt'
checkpoint_load_fn = 'latest.pt'
dtype = torch.bfloat16

# Mode
inference = False

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print ('device you will be isueing:', device)

device you will be isueing: cuda


## Set up logging
does not work yet

In [42]:
# logging
    # 2a4ae038692c0e045e311440fb0bd84bda2c7a
load_dotenv()
wandb_api_key = os.getenv('WANDB_API_KEY')
wandb_log = True
wandb_project = 'llm1'
wantd_run_name = 'llm1-'+datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
# 

wandb.login(key=wandb_api_key)
if wandb_log:
    import wandb
    wandb.init(project=wandb_project, name=wantd_run_name)

    

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


## Read Text

In [43]:
# some code
with open('wiki.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

print (text[30000:30300])


terms.
For example, there are objects in two groups (as shown on the right). The objects are various shapes, where one group has 3 of them while the other has 2. When the two groups combine into one, the overall amount (sum) of the shapes become 5.

Vertical Addition

The animation above demonstrate


### Set up tockenizer

In [44]:
# tokensizer
sp = spm.SentencePieceProcessor(model_file= 'wiki_tokenizer.model')
vocab_size = sp.get_piece_size()
print (f'Tokenizer vocab_size:  {vocab_size}')

Tokenizer vocab_size:  4096


## Set up encoder decoder

In [45]:
# encoder decoder
encode = lambda s: sp.Encode(s)
decode = lambda l: sp.Decode(l)
print (encode ('once upon a tine'))
print (decode (encode ('once upon a time')))

[2686, 698, 265, 261, 259, 500]
once upon a time


## load encoded data

In [46]:
if os.path.exists(f'encoded_data.pt'):
    data = torch.load('encoded_data.pt')
else:
    data = torch.tensor(encode(text), dtype=torch.long)
    torch.save(data,'encode_data.pt')

## Spint into train validation

In [47]:
data_size = len(data)
spl = int(0.9*data_size)
train_data = data[:spl]
val_data = data[spl:]

print (f'train data: {len(train_data)/1e6:2f} Million : validate data: {len(val_data)/1e6:2f}')

train data: 53.289969 Million : validate data: 5.921108


## Set up batch

In [48]:
# get batch
def get_batch(split):
    # BS = BatchSize XL = Sequence or context length
    data = train_data if split == 'train' else val_data
    inds = torch.randint(len(data)+context, (batch_size, ) )
    x = torch.stack([data[i: i+context] for i in inds]) # (BS, SL)
    y = torch.stack([data[i+1: i+context +1] for i in inds])

    x,y = x.to(device), y.to(device)
    return x,y


In [49]:
x, y = get_batch('train')
print (x.shape, y.shape)
print (x[0][:10])
print (y[0][:10])

torch.Size([8, 512]) torch.Size([8, 512])
tensor([ 293, 4052, 1024, 4060, 3406,  262,  954, 2847,  300, 3640],
       device='cuda:0')
tensor([4052, 1024, 4060, 3406,  262,  954, 2847,  300, 3640,  261],
       device='cuda:0')


## Coding LLM Architecture
19 Million parameter LLM
With 8 Batch Size 
with 128 batch Size will require 24 GB
Adjust Batch Size
Because of small data set and model results will be limite
demonstrate improvement during train


In [50]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.positions = nn.Embedding(context, embed_size)
        self.blocks = nn.Sequential(*[Block (n_heads) for _ in range (n_layers)] )
        self.ln = nn.LayerNorm(embed_size)
        self.final_linear = nn.Linear(embed_size, vocab_size, bias=BIAS)
        self.apply(self._init_weights)

    # parameter initializtion
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal(module.weight, mean= 0.00, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal(module.weight, mean = 0.0, std = 0.02)

    def forward(self, input, targets = None):
        loss = None
        BS, SL = input.shape
        emb = self.embeddings(input) #BS x XL x 284
        pos = self.positions(torch.arange(SL, device=device)) # SL x 384
        x = emb + pos
        x = self.blocks(x) # BS x SL x 384
        x = self.ln(x) # BS x SL x 384
        logits = self.final_linear(x) # sclaed number

        if targets is not None:
            BS, SL, VS = logits.shape
            logits = logits.view (BS*SL, VS)
            targets = targets.view(BS*SL)
            loss = F.cross_entropy(logits, targets)

            # manual calcualtion
            counts = logits.exp()
            prob = counts / counts.sum(1, keepdim=True)
            loss2 = - prob[torch.arange(BS*SL), targets].log().mean()

            if (not torch.allclose(loss, loss2)):
                print (f'(Loss diff) Pytorch: {loss.item()} manual: {loss2.item()}')

        return logits, loss
    
    def generate(self, input, max = 500):
        for _ in range (max):
            input = input[:,-context:] # (1, input lenght unte max of SL)
            logits, _ = self (input) # resutl (1, input lenght, 4096)
            logits = logits [:,-1,: ] # pick last probablity
            probs = F.softmax(logits, dim = -1)# # (1, 4096)
            next = torch.multinomial(probs, num_samples = 1)
            input = torch.cat((input, next), dim = 1)
        return input



In [51]:
class Block (nn.Module):
    def __init__(self, n_heads):
        super().__init__()
        head_size = embed_size // n_heads
        self.ma = Multihead(n_heads, head_size)
        self.feed_forward = ForwardLayer(embed_size)
        self.ln1 = nn.LayerNorm(embed_size)
        self.ln2 = nn.LayerNorm(embed_size)

    def forward (self, x):
        x = x + self.ma(self.ln1(x))
        x = x + self.feed_forward(self.ln2(x))
        return x





In [52]:
class ForwardLayer(nn.Module):
    def __init__(self, embed_size):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(embed_size,6*embed_size, bias= BIAS),
            nn.GELU(),
            nn.Linear(6*embed_size, embed_size, bias = BIAS),
            nn.Dropout(dropout)
    )
    def forward (self, x):
        x = self.network(x)
        return (x)
    

In [53]:
class Multihead(nn.Module):
    def __init__(self, n_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_heads)])
        self.combine = nn.Linear(head_size * n_heads, embed_size, bias = BIAS)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # x = torch.cat([Head(x) for head in self.heads], dim = 1)
        x = torch.cat([head(x) for head in self.heads], dim = -1)        # each  head ouputs (BS, SL, head_size)
        x = self.combine(x)
        x = self.dropout(x)
        return x 

In [54]:
class Head(nn.Module):
    def __init__ (self, head_size):
        super().__init__()
        self.queries = nn.Linear(embed_size, head_size, bias=BIAS)
        self.keys = nn.Linear(embed_size, head_size, bias=BIAS)
        self.values = nn.Linear(embed_size, head_size, bias=BIAS)

        self.register_buffer('trill', torch.tril(torch.ones(context,context)))
        self.dropout = nn.Dropout(dropout)

    def forward (self,x):
        BS, SL, VS = x.shape
        q=self.queries(x) #BS, SL, 54
        k=self.keys(x)
        v=self.values(x)

        attn_w = q @ k.transpose(-2, -1) * k.shape [-1]**-0.5 # BS, SL, SL
        attn_w = attn_w.masked_fill(self.trill[:SL, :SL]== 0, float ('-inf'))
        attn_w = F.softmax(attn_w, dim = -1) #BS, SL, SL

        x = attn_w @ v #BS, SL, SL
        return x


In [55]:
# # understand attention
# x,y = get_batch('train')
# print (f'shape of x, y {x.shape, y.shape}')
# x= x.to(device)
# y= y.to(device)
# head_size = embed_size // n_heads
# embeddings = nn.Embedding(vocab_size, embed_size).to(device)
# positions = nn.Embedding(context, embed_size).to(device)
# print(f'positions {positions}')
# queries = nn.Linear(embed_size, head_size, bias=BIAS).to(device)
# keys = nn.Linear(embed_size, head_size, bias=BIAS).to(device)
# values = nn.Linear(embed_size, head_size, bias=BIAS).to(device)
# tril = torch.tril(torch.ones(context,context)).to(device)

# emb = embeddings(x)
# # pos = positions(torch.arange(context, device =device))
# # print (f'pos {pos}')
# x = emb + pos

# q=queries(x)
# k=keys(x)
# v=values(x)
# print(f'shape of q, k, v {q.shape, k.shape, v.shape}')
# torch.set_printoptions(precision=2, sci_mode=False)
# print(q[0][0])

# attn_w = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
# # print (f'attn_w 1: {attn_w[:, :, 7]}')
# attn_w = attn_w.masked_fill(tril[:context, :context]==0, float('-inf'))
# # print (f'attn_w 2: {attn_w[:, :, 7]}')
# attn_w = F.softmax(attn_w, dim = -1)
# # print (f'attn_w 3: {attn_w[:, :, 7]}')
# x = attn_w @ v

In [56]:
# # understand attention matrix
# full = q @ k.transpose(-2, -1)
# a = q[0][5]
# b = k.transpose(-2, -1)[0,:,3]
# print (f'a, b: {a, b}')
# c = torch.dot (a, b)
# torch.set_printoptions(precision=2, sci_mode=False)
# print (f'c: {c:.2f}')
# print (f'full 0, 5, 3: {full[0][5][3]:.2f}')

In [57]:
# # underand th eupadtin of the V content
# # print(f'attn_w: {attn_w[:,:,7]} \nv: {v[:,:,7]}')
# print (f'attn_w and v shape: {attn_w.shape, v.shape}')

# print (f'x 07: {x[0][7]}')

# attn_scores2 = attn_w[0, 7, :] # Shape [512]
# # print (attn_scores2)
# result = torch.zeros (54)
# for i in range (54):
#     result[i] = torch.dot (attn_scores2, v[0, :, i])

# print (f'result: {result}')

In [58]:
%pdb


Automatic pdb calling has been turned ON


In [59]:
# test code
x, y = get_batch('train')
print (x.shape, y.shape)
print (x[0][:10])
print (y[0][:10])
model = GPT()
model = model.to(dtype = dtype)
model = model.to(device=device)


logits, loss = model (x, y)
print (f'loss: {loss.item()}')
# logits, loss, loss2 = model (x, y)
# # print (loss.item(), loss2.item())

torch.Size([8, 512]) torch.Size([8, 512])
tensor([4046, 2324, 2629, 3124,  314,  282,  820,  342,  324, 3412],
       device='cuda:0')
tensor([2324, 2629, 3124,  314,  282,  820,  342,  324, 3412, 4055],
       device='cuda:0')


C:\Users\Tiger\AppData\Local\Temp\ipykernel_43092\3126284424.py:18: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(module.weight, mean = 0.0, std = 0.02)
C:\Users\Tiger\AppData\Local\Temp\ipykernel_43092\3126284424.py:14: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(module.weight, mean= 0.00, std=0.02)


loss: 8.375


In [60]:
# Training Setup
model = GPT()
model = model.to(dtype)
model = model.to(device)

if compile:
    print ('Torch :: Compiling model')
    model = torch.compile(model)


sum(p.numel() for p in model.parameters())

C:\Users\Tiger\AppData\Local\Temp\ipykernel_43092\3126284424.py:18: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(module.weight, mean = 0.0, std = 0.02)
C:\Users\Tiger\AppData\Local\Temp\ipykernel_43092\3126284424.py:14: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  torch.nn.init.normal(module.weight, mean= 0.00, std=0.02)


19837954

In [15]:
@torch.no_grad()
def generate_sample(input):
    t1 = torch.tensor(encode (input), dtype = torch.long, device = device)
    t1 = t1[None, :] # (1, (ssixe of the ids))
    newgen = model.generate(t1, max = 64)[0].tolist()
    result = decode (newgen)
    print (f'{result}')

generate_sample('once upon a time')

once upon a time lead describedant exist year writer warekm eventsajorarsam dark record named ident�osedtain pres must Florida mar let websreme kept leavesutescomference white artic Western office houserysideroy Madoweverwapek collegeident div Seolkuff most presidentences again become Avajoreople stand� meansade�


In [ ]:
# code to test
# to create file